# Basic task sumbission

https://docs.canopy.umbra.space/reference/create_task

In [1]:
import requests
import os
import geopandas as gpd
import pandas as pd

In [2]:
# Sandbox does not work for searching feasibilities
API = "https://api.canopy.umbra.space"
SANDBOX_API = "https://api.canopy.prod.umbra-sandbox.space"

In [3]:
aoi = gpd.read_file("../paradise.geojson")
aoi.explore(zoom_start=12, marker_type='marker')#, marker_kwds=dict(icon=folium.map.Icon(color='black')))

## Check tasking potential acquitions ("feasability")

https://docs.canopy.umbra.space/docs/feasibility

In [19]:
# Search one week into the future
# Umbra suggests at least 90min into future to allow for scheduling
now = gpd.pd.Timestamp.now().floor('s') + gpd.pd.Timedelta(minutes=90)
next_week = now + gpd.pd.Timedelta(days=7)
# Umbra API requires Z at end (UTC)
print(now.isoformat()+'Z', next_week.isoformat()+'Z')

2024-10-18T16:44:36Z 2024-10-25T16:44:36Z


In [5]:
# Our quota is constrained to certain combinations of rangeResolutionMinMeters and multilookFactor
# for which there are recommended grazingAngle ranges
# https://docs.canopy.umbra.space/docs/suggested-tasking-parameters

# Pick one of these: 
# $675
acquisition_1_1 = {
    "rangeResolutionMinMeters": 1,
    "multilookFactor": 1,
    "grazingAngleMinDegrees": 42,
    "grazingAngleMaxDegrees": 80,
}

# $850
acquisition_1_2 = {
    "rangeResolutionMinMeters": 1,
    "multilookFactor": 2,
    "grazingAngleMinDegrees": 37,
    "grazingAngleMaxDegrees": 80,
}

# $1,200
acquisition_1_4= {
    "rangeResolutionMinMeters": 1,
    "multilookFactor": 4,
    "grazingAngleMinDegrees": 32,
    "grazingAngleMaxDegrees": 80,
}

# $3,250
acquisition_025_1= {
    "rangeResolutionMinMeters": 0.25,
    "multilookFactor": 1,
    "grazingAngleMinDegrees": 53,
    "grazingAngleMaxDegrees": 59,
}

acquitision = acquisition_1_1

In [6]:
url = f"{API}/tasking/feasibilities"
payload = {
    "imagingMode": "SPOTLIGHT",
    "spotlightConstraints": {
        "geometry": aoi.geometry[0].__geo_interface__,
        "polarization": "VV",
        "targetAzimuthAngleStartDegrees": 0,
        "targetAzimuthAngleEndDegrees": 360,
        "sceneSize": "5x5_KM"
    },
    "windowStartAt": now.isoformat()+'Z',
    "windowEndAt": next_week.isoformat()+'Z',
}

# Add acquitision constraints
payload['spotlightConstraints'].update(acquitision)
print(payload) 

headers = {
    "accept": "application/json",
    "content-type": "application/json",
    "authorization": f"Bearer {os.environ.get('UMBRA_API_TOKEN')}"
}

response = requests.post(url, json=payload, headers=headers)

{'imagingMode': 'SPOTLIGHT', 'spotlightConstraints': {'geometry': {'type': 'Point', 'coordinates': (-121.74456, 46.79679, 1589.0)}, 'polarization': 'VV', 'targetAzimuthAngleStartDegrees': 0, 'targetAzimuthAngleEndDegrees': 360, 'sceneSize': '5x5_KM', 'rangeResolutionMinMeters': 1, 'multilookFactor': 1, 'grazingAngleMinDegrees': 42, 'grazingAngleMaxDegrees': 80}, 'windowStartAt': '2024-10-18T11:41:23Z', 'windowEndAt': '2024-10-25T11:41:23Z'}


In [7]:
response.json()['id']

'67c2ddce-a4cc-43c3-b10b-540c07a7fc3a'

In [9]:
# Retrieve feasibilities

url = f"{API}/tasking/feasibilities/{response.json()['id']}"

response = requests.get(url, headers=headers)

df = pd.DataFrame(response.json()['opportunities'])
df

,windowStartAt,windowEndAt,durationSec,grazingAngleStartDegrees,grazingAngleEndDegrees,targetAzimuthAngleStartDegrees,targetAzimuthAngleEndDegrees,squintAngleStartDegrees,squintAngleEndDegrees,squintAngleEngineeringDegrees,slantRangeStartKm,slantRangeEndKm,groundRangeStartKm,groundRangeEndKm,satelliteId
0,2024-10-18T19:09:02+00:00,2024-10-18T19:19:59+00:00,57.0,57.733073,53.864118,320.476989,241.245853,145.132600,-135.360610,-55.132600,595.527085,621.212654,293.153284,340.019912,Umbra-08
1,2024-10-19T06:35:53+00:00,2024-10-19T06:46:39+00:00,46.0,42.099460,42.015011,245.084072,277.974628,16.024860,-16.468642,73.975140,836.131308,836.900914,569.477009,569.767330,Umbra-10
2,2024-10-19T18:50:56+00:00,2024-10-19T19:01:23+00:00,27.0,73.465889,73.755022,58.075849,145.709217,44.267646,-43.189908,45.732354,531.174482,530.718764,139.109772,138.741142,Umbra-08
3,2024-10-20T04:57:41+00:00,2024-10-20T05:08:52+00:00,71.0,52.737835,52.572742,214.245489,304.179565,44.565272,-44.854187,45.434728,635.631506,636.455191,357.449894,357.058623,Umbra-05
4,2024-10-20T17:59:53+00:00,2024-10-20T18:10:57+00:00,64.0,55.632369,55.792144,330.254314,241.156140,135.159976,-135.426598,-45.159976,621.487027,621.294783,322.923348,323.717100,Umbra-05
5,2024-10-21T18:47:26+00:00,2024-10-21T18:58:44+00:00,78.0,44.115160,42.187400,72.899578,132.168277,26.146677,-32.476643,63.853323,811.993734,837.705193,533.658138,570.017132,Umbra-09
6,2024-10-22T05:39:17+00:00,2024-10-22T05:50:31+00:00,74.0,42.069808,42.107580,98.361148,44.221969,152.685347,-152.905940,-62.685347,837.894943,836.648934,570.377139,567.548265,Umbra-09
7,2024-10-22T17:28:33+00:00,2024-10-22T17:39:45+00:00,72.0,50.333350,42.217241,77.499036,145.115861,22.260805,-44.803222,67.739195,661.266631,745.036758,390.020195,511.760832,Umbra-05
8,2024-10-22T18:53:10+00:00,2024-10-22T19:04:36+00:00,86.0,50.296677,44.727876,68.472464,145.544541,31.618192,-44.789638,58.381808,744.986062,805.092277,435.264124,525.530718,Umbra-09
9,2024-10-23T05:44:37+00:00,2024-10-23T05:56:30+00:00,113.0,42.291925,42.362541,116.743513,27.888946,135.243666,-135.435501,-45.243666,835.058512,832.988708,566.973674,562.425077,Umbra-09


In [10]:
df.iloc[0]

windowStartAt                     2024-10-18T19:09:02+00:00
windowEndAt                       2024-10-18T19:19:59+00:00
durationSec                                            57.0
grazingAngleStartDegrees                          57.733073
grazingAngleEndDegrees                            53.864118
targetAzimuthAngleStartDegrees                   320.476989
targetAzimuthAngleEndDegrees                     241.245853
squintAngleStartDegrees                            145.1326
squintAngleEndDegrees                            -135.36061
squintAngleEngineeringDegrees                      -55.1326
slantRangeStartKm                                595.527085
slantRangeEndKm                                  621.212654
groundRangeStartKm                               293.153284
groundRangeEndKm                                 340.019912
satelliteId                                        Umbra-08
Name: 0, dtype: object

In [11]:
df.to_csv('feasibilities-paradise-2024-10-18.csv')

In [14]:
# Add average incidence angle 
# For mountaines environment perfer smaller incidence angle
df['averageIncidenceAngle'] = 90 - (df['grazingAngleStartDegrees'] + df['grazingAngleEndDegrees']) / 2
cols = ['windowStartAt', 'durationSec','averageIncidenceAngle', 'targetAzimuthAngleStartDegrees', 'targetAzimuthAngleEndDegrees']
df[cols]

,windowStartAt,durationSec,averageIncidenceAngle,targetAzimuthAngleStartDegrees,targetAzimuthAngleEndDegrees
0,2024-10-18T19:09:02+00:00,57.0,34.201404,320.476989,241.245853
1,2024-10-19T06:35:53+00:00,46.0,47.942764,245.084072,277.974628
2,2024-10-19T18:50:56+00:00,27.0,16.389545,58.075849,145.709217
3,2024-10-20T04:57:41+00:00,71.0,37.344711,214.245489,304.179565
4,2024-10-20T17:59:53+00:00,64.0,34.287744,330.254314,241.156140
5,2024-10-21T18:47:26+00:00,78.0,46.848720,72.899578,132.168277
6,2024-10-22T05:39:17+00:00,74.0,47.911306,98.361148,44.221969
7,2024-10-22T17:28:33+00:00,72.0,43.724704,77.499036,145.115861
8,2024-10-22T18:53:10+00:00,86.0,42.487723,68.472464,145.544541
9,2024-10-23T05:44:37+00:00,113.0,47.672767,116.743513,27.888946


## Sandbox task

Rather than honing in on a particular acquisition from the above list, just use the same constraints as feasibility request

In [25]:
# NOTE: need different API key for sanbox mode?
url = f"{SANDBOX_API}/tasking/tasks"


payload.update({
    "tags": ["paradise","uw"],
    "taskName": "paradise1"
})

response = requests.post(url, json=payload, headers=headers)

print(response.text)
payload

{"id":"2fb57386-570f-4f79-85ac-42c8f029ab93","type":"Feature","geometry":{"type":"Point","coordinates":[-121.74456,46.79679,1589.0]},"properties":{"imagingMode":"SPOTLIGHT","spotlightConstraints":{"geometry":{"type":"Point","coordinates":[-121.74456,46.79679,1589.0]},"polarization":"VV","rangeResolutionMinMeters":1.0,"multilookFactor":1,"grazingAngleMinDegrees":42,"grazingAngleMaxDegrees":80,"targetAzimuthAngleStartDegrees":0,"targetAzimuthAngleEndDegrees":360,"sceneSize":"5x5_KM"},"windowStartAt":"2024-10-18T16:44:36+00:00","windowEndAt":"2024-10-25T16:44:36+00:00","deliveryConfigId":null,"productTypes":["GEC","SIDD","SICD","CPHD"],"tags":["paradise","uw"],"taskName":"paradise1","userOrderId":"","satelliteIds":["Umbra-05","Umbra-08","Umbra-09","Umbra-10"],"id":"2fb57386-570f-4f79-85ac-42c8f029ab93","createdAt":"2024-10-18T13:30:34.382414+00:00","updatedAt":"2024-10-18T13:30:34.382418+00:00","organizationId":"org_Eukdrsr1pe3dXZrR","status":"SUBMITTED","statusHistory":[{"status":"RECEIV

{'imagingMode': 'SPOTLIGHT',
 'spotlightConstraints': {'geometry': {'type': 'Point',
   'coordinates': [-121.74456, 46.79679, 1589]},
  'polarization': 'VV',
  'rangeResolutionMinMeters': 1,
  'multilookFactor': 1,
  'grazingAngleMinDegrees': 42,
  'grazingAngleMaxDegrees': 80,
  'targetAzimuthAngleStartDegrees': 0,
  'targetAzimuthAngleEndDegrees': 360,
  'sceneSize': '5x5_KM'},
 'windowStartAt': '2024-10-18T16:44:36Z',
 'windowEndAt': '2024-10-25T16:44:36Z',
 'tags': ['paradise', 'uw'],
 'taskName': 'paradise1'}

In [24]:
# Once submitted, check on status in Web UI 
# OR via API:

url = f"{SANDBOX_API}/tasking/tasks/{response.json()['id']}"

response = requests.get(url, headers=headers)

response.json()

{'id': 'ffe934dd-46f5-4ddd-92d6-2efa389de994',
 'type': 'Feature',
 'geometry': {'type': 'Point', 'coordinates': [-121.74456, 46.79679, 1589.0]},
 'properties': {'imagingMode': 'SPOTLIGHT',
  'spotlightConstraints': {'geometry': {'type': 'Point',
    'coordinates': [-121.74456, 46.79679, 1589.0]},
   'polarization': 'VV',
   'rangeResolutionMinMeters': 1.0,
   'multilookFactor': 1,
   'grazingAngleMinDegrees': 42,
   'grazingAngleMaxDegrees': 80,
   'targetAzimuthAngleStartDegrees': 0,
   'targetAzimuthAngleEndDegrees': 360,
   'sceneSize': '5x5_KM'},
  'windowStartAt': '2024-10-18T16:44:36+00:00',
  'windowEndAt': '2024-10-25T16:44:36+00:00',
  'deliveryConfigId': None,
  'productTypes': ['GEC', 'SIDD', 'SICD', 'CPHD'],
  'tags': ['paradise'],
  'taskName': 'paradise1',
  'userOrderId': '',
  'satelliteIds': ['Umbra-05', 'Umbra-08', 'Umbra-09', 'Umbra-10'],
  'id': 'ffe934dd-46f5-4ddd-92d6-2efa389de994',
  'createdAt': '2024-10-18T13:21:14.000943+00:00',
  'updatedAt': '2024-10-18T13:

## Actually submit task!

In [30]:
# NOTE: need different API key for sanbox mode?
url = f"{API}/tasking/tasks"
response = requests.post(url, json=payload, headers=headers)
print(response.text)

{"id":"380908ae-be21-476d-8a0e-f5c177e8107e","type":"Feature","geometry":{"type":"Point","coordinates":[-121.74456,46.79679,1589.0]},"properties":{"imagingMode":"SPOTLIGHT","spotlightConstraints":{"geometry":{"type":"Point","coordinates":[-121.74456,46.79679,1589.0]},"polarization":"VV","rangeResolutionMinMeters":1.0,"multilookFactor":1,"grazingAngleMinDegrees":42,"grazingAngleMaxDegrees":80,"targetAzimuthAngleStartDegrees":0,"targetAzimuthAngleEndDegrees":360,"sceneSize":"5x5_KM"},"windowStartAt":"2024-10-18T16:44:36+00:00","windowEndAt":"2024-10-25T16:44:36+00:00","deliveryConfigId":null,"productTypes":["GEC","SIDD","SICD","CPHD"],"tags":["paradise","uw"],"taskName":"paradise1","userOrderId":"","satelliteIds":["Umbra-05","Umbra-08","Umbra-09","Umbra-10"],"id":"380908ae-be21-476d-8a0e-f5c177e8107e","createdAt":"2024-10-18T13:32:51.204523+00:00","updatedAt":"2024-10-18T13:32:51.204530+00:00","organizationId":"org_Eukdrsr1pe3dXZrR","status":"SUBMITTED","statusHistory":[{"status":"RECEIV

In [31]:

# Check task status
url = f"{API}/tasking/tasks/{response.json()['id']}"
response = requests.get(url, headers=headers)
response.json()

{'id': '380908ae-be21-476d-8a0e-f5c177e8107e',
 'type': 'Feature',
 'geometry': {'type': 'Point', 'coordinates': [-121.74456, 46.79679, 1589.0]},
 'properties': {'imagingMode': 'SPOTLIGHT',
  'spotlightConstraints': {'geometry': {'type': 'Point',
    'coordinates': [-121.74456, 46.79679, 1589.0]},
   'polarization': 'VV',
   'rangeResolutionMinMeters': 1.0,
   'multilookFactor': 1,
   'grazingAngleMinDegrees': 42,
   'grazingAngleMaxDegrees': 80,
   'targetAzimuthAngleStartDegrees': 0,
   'targetAzimuthAngleEndDegrees': 360,
   'sceneSize': '5x5_KM'},
  'windowStartAt': '2024-10-18T16:44:36+00:00',
  'windowEndAt': '2024-10-25T16:44:36+00:00',
  'deliveryConfigId': None,
  'productTypes': ['GEC', 'SIDD', 'SICD', 'CPHD'],
  'tags': ['paradise', 'uw'],
  'taskName': 'paradise1',
  'userOrderId': '',
  'satelliteIds': ['Umbra-05', 'Umbra-08', 'Umbra-09', 'Umbra-10'],
  'id': '380908ae-be21-476d-8a0e-f5c177e8107e',
  'createdAt': '2024-10-18T13:32:51.204523+00:00',
  'updatedAt': '2024-10-